# L-DOPA Synthesis in _E. coli_: Review, Analysis, and Future Directions
#### Russell O. Stewart
#### ENGS 161
#### 20W
#### Final Project

# Road Map

* Introduction
    * Background
    * Previous Synthetic Attempts
* Resources and Methods
* Analysis
    * Black Box Stoichiometry
    * Flux Balance Analysis
    * Balance Double-Check
    * Thermodynamics
* Conclusions and Future Directions
    * Re-Evaluation of Previous Synthetic Attempts
    * Future Directions

# Background

# L-DOPA, Derivatives and their Pharmacologic Applications

![Image](external_figures/l_dopa_derivatives.png)

# Synthesis in _E. coli_: The Shikimate Pathway
![Image](external_figures/Shikimate-pathway-modified-from-Bentley-1990-Schmid-and-Amrhein-1995.png)

# Previous Biosynthetic Attempts in _E. coli_

# Munoz et al, 2011
![Image](external_figures/munoz_shikimate_pathway.png)

# Wei et al, 2016

![Image](external_figures/wei_synthetic_pathway.png)

# Resources and Methods

![Image](figures/bigg_map.png)

# Black-Box Stoichiometry

* **_What is the maximum theoretical yield of L-DOPA from glucose?_**
* **_What are feasible sources of nitrogen for our synthesis?_**

Obtain a stoichiometric matrix $S$ of glucose, L-DOPA, and $H_2O$, $NH_3$, and $CO_2$

Define our flux vector $q$

$Sq=0$

$q_{calc}=-S_{calc}^{-1}S_{meas}q_{meas}$

Measured flux: glucose (-1). Unmeasured fluxes: all others.

In [5]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
solution = pd.read_csv('FBA_results/aerobic.csv' , index_col = 0)
solution_glu = pd.read_csv('FBA_results/aerobic_glutamate.csv' , index_col = 0)
solution_exchanges = solution[solution.index.str.contains('EX_')]
solution_glu_exchanges = solution_glu[solution_glu.index.str.contains('EX_')]
solution_exchanges = solution_exchanges.query('abs(fluxes) > .0000000001')
solution_glu_exchanges = solution_glu_exchanges.query('abs(fluxes) > .0000000001')
solution_index = [re.search(r"_[^_]+_{1}",i).group() for i in solution_exchanges.index]
solution_index = [i[1:len(i)-1] for i in solution_index]
solution_exchanges.index = solution_index
solution_glu_index = [re.search(r"_[^_]+_{1}",i).group() for i in solution_glu_exchanges.index]
solution_glu_index = [i[1:len(i)-1] for i in solution_glu_index]
solution_glu_exchanges.index = solution_glu_index
S = pd.read_excel('exchange_stoichiometries.xls' , index_col = 0)
print('Stoichiometric Matrix')
display(S)

Stoichiometric Matrix


,co2,for,glc,glu,h,h2o,o2,levdopa,nh4
c,1,1,6,5,0,0,0,9,0
h,0,1,12,8,1,2,0,11,4
o,2,2,6,4,0,1,2,4,0
n,0,0,0,1,0,0,0,1,1


# Glucose as Carbon Source, Ammonia as Nitrogen Source

In [3]:
S_meas = S[['glc']]
S_calc = S[['h2o','nh4','levdopa','co2']]
S_calc.at['h','nh4'] = 3#Use ammonia, not ammonium!
S_calc.columns = ['h2o','nh3','levdopa','co2']
v_meas = pd.DataFrame(np.array([-1]) , index = ['glc'])
v_calc = -1* np.linalg.inv(S_calc) @ S_meas @ v_meas
v_calc.index = ['h2o','nh3','levdopa','co2']
v_calc.columns = ['Flux']
v_calc

,Flux
h2o,3.333333
nh3,-0.666667
levdopa,0.666667
co2,0.000000


$3\ Glucose + 2\ NH_3 \rightarrow 10\ H_2O + 2\ levdopa$.

$Y_{ldopa/glu}=0.667\ mol/mol=1\ cmol/cmol$

$Y_{ldopa/NH3}=1\ mol/mol$

# Glucose as Carbon Source, Glutamate as Nitrogen Source

In [6]:
S_meas = S[['glc']]
S_calc = S[['h2o','glu','levdopa','co2']]
S_calc.at['h','glu'] = S_calc.at['h','glu'] + 1#Use glutamic acid, not glutamate!
S_calc.columns = ['h2o','nh3','levdopa','co2']
v_meas = pd.DataFrame(np.array([-1]) , index = ['glc'])
v_calc = -1* np.linalg.inv(S_calc) @ S_meas @ v_meas
v_calc.index = ['h2o','glu','levdopa','co2']
v_calc.columns = ['Flux']
v_calc

,Flux
h2o,4.666667
glu,-1.333333
levdopa,1.333333
co2,0.666667


$3\ Glucose + 4\ Glu \rightarrow 2\ CO_2 + 14\ H_2O + 4\ levdopa$.

$Y_{ldopa/glu}=1.33\ mol/mol$

$Y_{ldopa/total\ carbon\ input}=0.947\ cmol/cmol$

$Y_{ldopa/glu}=1\ mol/mol$

# Flux Balance Analysis

* **_On which pathways will this synthesis depend?_**
* **_Does our maximum yield drop at all?_**
* **_Does this inform our growth conditions?_**

Run flux balance analysis using `cobra`, and display our results using `escher`

In [7]:
#load up everything
%matplotlib inline
import cobra
from cobra import Model, Reaction, Metabolite
cobra_config = cobra.Configuration()
import escher
from escher import Builder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import sleep
import cobra.test
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,double_reaction_deletion)

network = pd.read_csv('shikimate_reactions.csv' , header=0)#shikimate related reactions to add
sinks = pd.read_csv('sinks.csv' , header=0)#sinks to add

levdopa_synthesis = cobra.io.load_json_model("e_coli_core.json")#e coli central metabolism model from BiGG
#levdopa_synthesis.solver = 'cplex'

#define intracellular reactions not in central metabolism model
for (myenzyme , mypathway , myreaction) in network.itertuples(index = False):
    rxn = Reaction(myenzyme)
    levdopa_synthesis.add_reactions([rxn])
    rxn.name = myenzyme
    rxn.subsystem = mypathway
    rxn.build_reaction_from_string(myreaction)

#define extra transport reactions not in central metabolism model
for (mymetabolite , mydirection) in sinks.itertuples(index = False):
    myreaction = '%s -->' % mymetabolite
    rxn = Reaction('EX_%s' % mymetabolite)
    levdopa_synthesis.add_reactions([rxn])
    rxn.subsystem = 'Extracellular exchange'
    rxn.name = '%s_transport' % mymetabolite
    rxn.build_reaction_from_string(myreaction)
    rxn.lower_bound = 0.
    rxn.upper_bound = 1000.

unknown metabolite 'dahp_c' created
unknown metabolite 'dhq_c' created
unknown metabolite '3dhs_c' created
unknown metabolite 'shikimate_c' created
unknown metabolite 'shikimate3p_c' created
unknown metabolite 'epsp_c' created
unknown metabolite 'chorismate_c' created
unknown metabolite 'prephenate_c' created
unknown metabolite '4hppa_c' created
unknown metabolite 'tyr__L_c' created
unknown metabolite 'levdopa_c' created
unknown metabolite 'levdopa_e' created


In [9]:
levdopa_synthesis.objective = 'EX_levdopa_e'#maximize L-DOPA production
builder = Builder(map_json = 'full_map.json')
builder.model = levdopa_synthesis
builder.highlight_missing = True
solution = builder.model.optimize()
builder.reaction_data = solution.fluxes/ abs(solution['EX_glc__D_e'])
Y_ldopa_glc = solution['EX_levdopa_e']/abs(solution['EX_glc__D_e'])
Y_ldopa_nh4 = solution['EX_levdopa_e']/abs(solution['EX_nh4_e'])
print('Yield of levdopa from glucose: %f mol/mol; %f cmol/cmol' % (Y_ldopa_glc , 9/6*Y_ldopa_glc))
print('Yield of levdopa from ammonia: %f mol/mol' % Y_ldopa_nh4)

Yield of levdopa from glucose: 0.508230 mol/mol; 0.762345 cmol/cmol
Yield of levdopa from ammonia: 1.000000 mol/mol


![Image](Escher_Maps/aerobic_fba.png)